# Multigrid Method for Solving Elliptic PDEs

## Introduction

The **Multigrid Method** is one of the most efficient algorithms for solving large sparse linear systems arising from the discretization of elliptic partial differential equations. Unlike classical iterative methods (Jacobi, Gauss-Seidel) which exhibit slow convergence for low-frequency error components, multigrid methods achieve optimal $O(N)$ complexity by operating on multiple grid resolutions simultaneously.

## Mathematical Foundation

### Model Problem: Poisson Equation

Consider the 2D Poisson equation on a unit square $\Omega = [0,1]^2$:

$$-\nabla^2 u = -\left(\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}\right) = f(x,y)$$

with Dirichlet boundary conditions $u = 0$ on $\partial\Omega$.

### Finite Difference Discretization

Using a uniform grid with spacing $h = 1/(n+1)$ and the standard 5-point stencil:

$$\frac{-u_{i-1,j} - u_{i+1,j} - u_{i,j-1} - u_{i,j+1} + 4u_{i,j}}{h^2} = f_{i,j}$$

This yields the linear system:

$$A_h u_h = f_h$$

where $A_h$ is the discrete Laplacian matrix.

## Why Classical Iterative Methods Are Slow

### Error Decomposition

The error $e^{(k)} = u - u^{(k)}$ after $k$ iterations can be decomposed into Fourier modes:

$$e^{(k)}_{i,j} = \sum_{p,q} \hat{e}^{(k)}_{p,q} \sin(p\pi x_i)\sin(q\pi y_j)$$

### Smoothing Property

Classical relaxation methods (e.g., Gauss-Seidel) efficiently damp **high-frequency** error components but struggle with **low-frequency** components. The damping factor for mode $(p,q)$ is:

$$\rho_{p,q} = 1 - \frac{\sin^2(p\pi h/2) + \sin^2(q\pi h/2)}{2}$$

For low frequencies ($p,q \ll 1/h$), $\rho_{p,q} \approx 1$, leading to slow convergence.

## Multigrid Strategy

The key insight: **low-frequency errors on a fine grid appear as high-frequency errors on a coarser grid**.

### Two-Grid Algorithm

1. **Pre-smoothing**: Apply $\nu_1$ relaxation sweeps on fine grid $\Omega_h$
2. **Compute residual**: $r_h = f_h - A_h u_h$
3. **Restrict**: Transfer residual to coarse grid $r_H = I_h^H r_h$
4. **Coarse grid solve**: Solve $A_H e_H = r_H$ (exactly or recursively)
5. **Interpolate**: Prolongate correction $e_h = I_H^h e_H$
6. **Correct**: $u_h \leftarrow u_h + e_h$
7. **Post-smoothing**: Apply $\nu_2$ relaxation sweeps

### Grid Transfer Operators

**Restriction** (fine to coarse) using full weighting:

$$r^H_{i,j} = \frac{1}{16}\left[r^h_{2i-1,2j-1} + r^h_{2i+1,2j-1} + r^h_{2i-1,2j+1} + r^h_{2i+1,2j+1} + 2(r^h_{2i,2j-1} + r^h_{2i,2j+1} + r^h_{2i-1,2j} + r^h_{2i+1,2j}) + 4r^h_{2i,2j}\right]$$

**Prolongation** (coarse to fine) using bilinear interpolation:

$$e^h_{2i,2j} = e^H_{i,j}, \quad e^h_{2i+1,2j} = \frac{1}{2}(e^H_{i,j} + e^H_{i+1,j})$$

### V-Cycle and W-Cycle

- **V-Cycle**: Single recursive call on each level
- **W-Cycle**: Two recursive calls on each level
- **Full Multigrid (FMG)**: Nested iteration starting from coarsest grid

## Convergence Analysis

The multigrid convergence rate is **grid-independent**:

$$\|e^{(k+1)}\| \leq \rho_{MG} \|e^{(k)}\|$$

where $\rho_{MG} \approx 0.1$ is independent of $h$. This yields:

- **Work per V-cycle**: $O(N)$
- **Iterations for convergence**: $O(1)$
- **Total complexity**: $O(N)$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

# Set random seed for reproducibility
np.random.seed(42)

## Implementation

We implement a 2D multigrid solver for the Poisson equation with the following components:

1. Gauss-Seidel smoother (red-black ordering for efficiency)
2. Full weighting restriction operator
3. Bilinear interpolation prolongation operator
4. V-cycle multigrid iteration

In [ ]:
def gauss_seidel_red_black(u, f, h, num_sweeps=1):
    """
    Red-Black Gauss-Seidel smoother for 2D Poisson equation.
    
    Parameters:
    -----------
    u : ndarray
        Current solution estimate (modified in place)
    f : ndarray
        Right-hand side
    h : float
        Grid spacing
    num_sweeps : int
        Number of Gauss-Seidel sweeps
    """
    n = u.shape[0] - 2  # Interior points
    h2 = h * h
    
    for _ in range(num_sweeps):
        # Red points (i+j even)
        for i in range(1, n+1):
            for j in range(1, n+1):
                if (i + j) % 2 == 0:
                    u[i, j] = 0.25 * (u[i-1, j] + u[i+1, j] + 
                                      u[i, j-1] + u[i, j+1] + h2 * f[i, j])
        
        # Black points (i+j odd)
        for i in range(1, n+1):
            for j in range(1, n+1):
                if (i + j) % 2 == 1:
                    u[i, j] = 0.25 * (u[i-1, j] + u[i+1, j] + 
                                      u[i, j-1] + u[i, j+1] + h2 * f[i, j])


def compute_residual(u, f, h):
    """
    Compute residual r = f - Au for 2D Poisson equation.
    """
    n = u.shape[0] - 2
    r = np.zeros_like(u)
    h2 = h * h
    
    for i in range(1, n+1):
        for j in range(1, n+1):
            r[i, j] = f[i, j] - (4*u[i, j] - u[i-1, j] - u[i+1, j] 
                                 - u[i, j-1] - u[i, j+1]) / h2
    return r


def restrict_full_weighting(r_fine):
    """
    Restrict from fine grid to coarse grid using full weighting.
    """
    n_fine = r_fine.shape[0] - 2
    n_coarse = n_fine // 2
    r_coarse = np.zeros((n_coarse + 2, n_coarse + 2))
    
    for i in range(1, n_coarse + 1):
        for j in range(1, n_coarse + 1):
            i_f, j_f = 2*i, 2*j
            r_coarse[i, j] = (1/16) * (
                r_fine[i_f-1, j_f-1] + r_fine[i_f+1, j_f-1] + 
                r_fine[i_f-1, j_f+1] + r_fine[i_f+1, j_f+1] +
                2 * (r_fine[i_f, j_f-1] + r_fine[i_f, j_f+1] + 
                     r_fine[i_f-1, j_f] + r_fine[i_f+1, j_f]) +
                4 * r_fine[i_f, j_f]
            )
    return r_coarse


def prolongate_bilinear(e_coarse, n_fine):
    """
    Prolongate from coarse grid to fine grid using bilinear interpolation.
    """
    e_fine = np.zeros((n_fine + 2, n_fine + 2))
    n_coarse = e_coarse.shape[0] - 2
    
    for i in range(1, n_coarse + 1):
        for j in range(1, n_coarse + 1):
            i_f, j_f = 2*i, 2*j
            
            # Direct injection
            e_fine[i_f, j_f] = e_coarse[i, j]
            
            # Interpolate horizontally
            if i < n_coarse:
                e_fine[i_f+1, j_f] = 0.5 * (e_coarse[i, j] + e_coarse[i+1, j])
            
            # Interpolate vertically
            if j < n_coarse:
                e_fine[i_f, j_f+1] = 0.5 * (e_coarse[i, j] + e_coarse[i, j+1])
            
            # Interpolate diagonally
            if i < n_coarse and j < n_coarse:
                e_fine[i_f+1, j_f+1] = 0.25 * (e_coarse[i, j] + e_coarse[i+1, j] +
                                               e_coarse[i, j+1] + e_coarse[i+1, j+1])
    
    return e_fine

In [ ]:
def v_cycle(u, f, h, level, max_levels, nu1=2, nu2=2):
    """
    V-cycle multigrid iteration.
    
    Parameters:
    -----------
    u : ndarray
        Current solution estimate
    f : ndarray
        Right-hand side
    h : float
        Grid spacing
    level : int
        Current grid level (0 = finest)
    max_levels : int
        Maximum number of levels
    nu1 : int
        Number of pre-smoothing sweeps
    nu2 : int
        Number of post-smoothing sweeps
    """
    n = u.shape[0] - 2
    
    # Coarsest level: solve directly
    if level == max_levels - 1 or n <= 2:
        # Use many smoothing iterations as direct solver
        gauss_seidel_red_black(u, f, h, num_sweeps=50)
        return
    
    # Pre-smoothing
    gauss_seidel_red_black(u, f, h, num_sweeps=nu1)
    
    # Compute residual
    r = compute_residual(u, f, h)
    
    # Restrict residual to coarse grid
    r_coarse = restrict_full_weighting(r)
    
    # Initialize coarse grid error to zero
    n_coarse = n // 2
    e_coarse = np.zeros((n_coarse + 2, n_coarse + 2))
    
    # Recursive call on coarse grid
    v_cycle(e_coarse, r_coarse, 2*h, level + 1, max_levels, nu1, nu2)
    
    # Prolongate and correct
    e_fine = prolongate_bilinear(e_coarse, n)
    u += e_fine
    
    # Post-smoothing
    gauss_seidel_red_black(u, f, h, num_sweeps=nu2)


def multigrid_solve(f, h, max_levels, tol=1e-10, max_iter=100):
    """
    Solve Au = f using multigrid V-cycles.
    
    Returns:
    --------
    u : ndarray
        Solution
    residuals : list
        Residual norm history
    """
    n = f.shape[0] - 2
    u = np.zeros_like(f)
    
    residuals = []
    
    for iteration in range(max_iter):
        v_cycle(u, f, h, 0, max_levels)
        
        r = compute_residual(u, f, h)
        res_norm = np.linalg.norm(r[1:-1, 1:-1])
        residuals.append(res_norm)
        
        if res_norm < tol:
            break
    
    return u, residuals

## Test Problem

We solve the Poisson equation with a manufactured solution:

$$u(x,y) = \sin(\pi x)\sin(\pi y)$$

which gives:

$$f(x,y) = 2\pi^2 \sin(\pi x)\sin(\pi y)$$

In [ ]:
def solve_poisson_multigrid(n, max_levels=5):
    """
    Solve 2D Poisson equation using multigrid.
    
    Parameters:
    -----------
    n : int
        Number of interior grid points in each direction
    max_levels : int
        Maximum number of grid levels
    """
    h = 1.0 / (n + 1)
    
    # Create grid
    x = np.linspace(0, 1, n + 2)
    y = np.linspace(0, 1, n + 2)
    X, Y = np.meshgrid(x, y, indexing='ij')
    
    # Exact solution
    u_exact = np.sin(np.pi * X) * np.sin(np.pi * Y)
    
    # Right-hand side
    f = 2 * np.pi**2 * np.sin(np.pi * X) * np.sin(np.pi * Y)
    
    # Apply boundary conditions (already zero)
    
    # Solve using multigrid
    u, residuals = multigrid_solve(f, h, max_levels)
    
    # Compute error
    error = np.max(np.abs(u - u_exact))
    
    return u, u_exact, residuals, error, X, Y


# Solve for different grid sizes
grid_sizes = [16, 32, 64, 128]
errors = []
all_residuals = {}

for n in grid_sizes:
    u, u_exact, residuals, error, X, Y = solve_poisson_multigrid(n, max_levels=5)
    errors.append(error)
    all_residuals[n] = residuals
    print(f"Grid {n}x{n}: Max error = {error:.6e}, Iterations = {len(residuals)}")

## Comparison with Gauss-Seidel

To demonstrate the superior convergence of multigrid, we compare it with pure Gauss-Seidel iteration.

In [ ]:
def gauss_seidel_solve(f, h, tol=1e-10, max_iter=10000):
    """
    Solve Au = f using pure Gauss-Seidel iteration.
    """
    n = f.shape[0] - 2
    u = np.zeros_like(f)
    
    residuals = []
    
    for iteration in range(max_iter):
        gauss_seidel_red_black(u, f, h, num_sweeps=1)
        
        r = compute_residual(u, f, h)
        res_norm = np.linalg.norm(r[1:-1, 1:-1])
        residuals.append(res_norm)
        
        if res_norm < tol:
            break
    
    return u, residuals


# Compare convergence for n=64
n = 64
h = 1.0 / (n + 1)

x = np.linspace(0, 1, n + 2)
y = np.linspace(0, 1, n + 2)
X, Y = np.meshgrid(x, y, indexing='ij')

f = 2 * np.pi**2 * np.sin(np.pi * X) * np.sin(np.pi * Y)

# Multigrid
_, mg_residuals = multigrid_solve(f, h, max_levels=5, max_iter=20)

# Gauss-Seidel (limit iterations for comparison)
_, gs_residuals = gauss_seidel_solve(f, h, max_iter=500)

print(f"Multigrid iterations: {len(mg_residuals)}")
print(f"Gauss-Seidel iterations (capped at 500): {len(gs_residuals)}")
print(f"Final MG residual: {mg_residuals[-1]:.6e}")
print(f"Final GS residual: {gs_residuals[-1]:.6e}")

## Visualization

In [ ]:
fig = plt.figure(figsize=(14, 10))

# Plot 1: Convergence comparison (MG vs GS)
ax1 = fig.add_subplot(2, 2, 1)
ax1.semilogy(range(1, len(mg_residuals) + 1), mg_residuals, 'b-o', 
             linewidth=2, markersize=6, label='Multigrid V-cycle')
ax1.semilogy(range(1, len(gs_residuals) + 1), gs_residuals, 'r-', 
             linewidth=1.5, alpha=0.7, label='Gauss-Seidel')
ax1.set_xlabel('Iteration', fontsize=12)
ax1.set_ylabel('Residual Norm', fontsize=12)
ax1.set_title('Convergence: Multigrid vs Gauss-Seidel (64×64 grid)', fontsize=12)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: Grid-independent convergence
ax2 = fig.add_subplot(2, 2, 2)
colors = plt.cm.viridis(np.linspace(0, 0.8, len(grid_sizes)))
for i, n in enumerate(grid_sizes):
    res = all_residuals[n]
    # Normalize by initial residual
    res_normalized = np.array(res) / res[0] if res[0] > 0 else res
    ax2.semilogy(range(1, len(res) + 1), res_normalized, 
                 color=colors[i], linewidth=2, marker='o', markersize=4,
                 label=f'{n}×{n}')
ax2.set_xlabel('V-cycle', fontsize=12)
ax2.set_ylabel('Relative Residual', fontsize=12)
ax2.set_title('Grid-Independent Convergence Rate', fontsize=12)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

# Plot 3: Discretization error (order of accuracy)
ax3 = fig.add_subplot(2, 2, 3)
h_values = [1.0/(n+1) for n in grid_sizes]
ax3.loglog(h_values, errors, 'ko-', linewidth=2, markersize=8, label='Computed error')
# Reference line for O(h^2)
h_ref = np.array(h_values)
ref_line = errors[0] * (h_ref / h_values[0])**2
ax3.loglog(h_ref, ref_line, 'r--', linewidth=1.5, label='$O(h^2)$ reference')
ax3.set_xlabel('Grid spacing $h$', fontsize=12)
ax3.set_ylabel('Max error $\\|u - u_h\\|_\\infty$', fontsize=12)
ax3.set_title('Second-Order Discretization Accuracy', fontsize=12)
ax3.legend(fontsize=10)
ax3.grid(True, alpha=0.3)

# Plot 4: Solution visualization
ax4 = fig.add_subplot(2, 2, 4)
# Solve on finest grid for visualization
n_vis = 64
u_vis, u_exact_vis, _, _, X_vis, Y_vis = solve_poisson_multigrid(n_vis)
contour = ax4.contourf(X_vis, Y_vis, u_vis, levels=20, cmap='RdBu_r')
plt.colorbar(contour, ax=ax4, label='$u(x,y)$')
ax4.set_xlabel('$x$', fontsize=12)
ax4.set_ylabel('$y$', fontsize=12)
ax4.set_title('Solution: $-\\nabla^2 u = 2\\pi^2 \\sin(\\pi x)\\sin(\\pi y)$', fontsize=12)
ax4.set_aspect('equal')

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to 'plot.png'")

## Computational Complexity Analysis

The work at each grid level in a V-cycle:

$$W = N + \frac{N}{4} + \frac{N}{16} + \cdots = N\sum_{k=0}^{\infty}\frac{1}{4^k} = \frac{4N}{3}$$

Since the convergence rate is independent of $N$, we achieve **optimal $O(N)$ complexity**.

### Comparison of Complexities

| Method | Complexity | Iterations for $\epsilon$ accuracy |
|--------|------------|------------------------------------|
| Jacobi/Gauss-Seidel | $O(N^2)$ | $O(N)$ |
| Conjugate Gradient | $O(N^{3/2})$ | $O(N^{1/2})$ |
| **Multigrid** | $O(N)$ | $O(1)$ |

## Conclusion

The multigrid method achieves optimal computational complexity for elliptic PDEs by exploiting the complementary smoothing properties of relaxation methods across different grid resolutions. The key insights are:

1. **Smoothing**: Relaxation methods efficiently eliminate high-frequency errors
2. **Coarse grid correction**: Low-frequency errors appear as high-frequency on coarser grids
3. **Grid independence**: Convergence rate is independent of problem size

This makes multigrid the method of choice for large-scale problems in computational science and engineering, including fluid dynamics, structural mechanics, and electromagnetic simulations.